In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "Test1234"
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
#Set up data table, map, and bar chart with pagnation, sorting ability, and selection
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div([
        html.Img(src='assets/GraziosoSalvareLogo.png', height='80px'),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Cody Adams'))),
    html.Hr(),
        
    html.Div([
        html.Label("Select Rescue Type:"),
        dcc.RadioItems(
        id='rescue-type-filter',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
            {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'block'}
        )
    ]),

    html.Hr(),
        
    
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        style_cell={
        'textAlign': 'left',
        'minWidth': '100px',
        'width': '100px',
        'maxWidth': '180px',
        'whiteSpace': 'normal'
        }

    ),
    #Set up map and chart to be centered under the table
    html.Br(),
     html.Div([
    html.Div(
        id='map-id',
        style={
            'display': 'flex',
            'justifyContent': 'center',
            'marginBottom': '20px'
        }
    ),
    html.Div(
        dcc.Graph(id='bar-chart-id'),
        style={
            'display': 'flex',
            'justifyContent': 'center'
        }
    )
])


])


    ])

    


#############################################
# Interaction Between Components / Controller
#############################################
    

#Filter Callback: Rescue Type filters the table
    
@app.callback(
    Output('datatable-id', 'data'),
    Input('rescue-type-filter', 'value')
)
def filter_table(rescue_type):
    if rescue_type == 'Reset':
        filtered_data = shelter.read({})
    elif rescue_type == 'Water Rescue':
        filtered_data = shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        })
    elif rescue_type == 'Mountain or Wilderness Rescue':
        filtered_data = shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        })
    elif rescue_type == 'Disaster or Individual Tracking':
        filtered_data = shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        })
    else:
        filtered_data = shelter.read({})

    for doc in filtered_data:
        doc.pop('_id', None)
    return filtered_data
    
    
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
#FIXME Add in the code for your geolocation chart
     # If there's no data, return a blank map
    if viewData is None or len(viewData) == 0:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                   center=[30.75, -97.48], zoom=10, children=[
                       dl.TileLayer(id="base-layer-id")
                   ])
        ]

    dff = pd.DataFrame.from_dict(viewData)

    # Default to first row if no row is selected
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]

    # Additional safety check for index bounds
    if row >= len(dff.index) or 13 not in range(len(dff.columns)):
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                   center=[30.75, -97.48], zoom=10, children=[
                       dl.TileLayer(id="base-layer-id")
                   ])
        ]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(position=[dff.iloc[row,13], dff.iloc[row,14]],
                             children=[
                                 dl.Tooltip(dff.iloc[row,4]),
                                 dl.Popup([
                                     html.H1("Animal Name"),
                                     html.P(dff.iloc[row,9])
                                 ])
                             ])
               ])
    ]
    
#Bar chart showing the top breeds
@app.callback(
    Output('bar-chart-id', 'figure'),
    Input('datatable-id', 'data')
)
def update_bar_chart(data):
    if data is None or len(data) == 0:
        return px.bar(title="No Data Available")

    dff = pd.DataFrame(data)
    breed_counts = dff['breed'].value_counts().nlargest(10).reset_index()
    breed_counts.columns = ['Breed', 'Count']

    fig = px.bar(breed_counts, x='Breed', y='Count', title="Top 10 Dog Breeds by Count")
    return fig
    

app.run_server(mode='external', debug=True, port=8050)


Dash app running on http://127.0.0.1:8050/
